<a href="https://colab.research.google.com/github/Rahma-Farag/langchain_project/blob/main/RAG_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q torch transformers transformers chromadb accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille

In [11]:
from google.colab import userdata
# userdata.get('HF_API')

In [12]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_API')

In [13]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Get the device ID of the current GPU
    device_id = torch.cuda.current_device()
    print(f"Current GPU device ID: {device_id}")
else:
    print("CUDA (GPU) is not available.")


Current GPU device ID: 0


In [14]:
# # !pip install -q langchain==0.0.219
# # !pip install langchain-huggingface
# !pip install -q chromadb==0.3.26
# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

# !pip install -q python-dotenv==1.0.0

# !pip install -q pandas==1.5.3
# !pip install -q unstructured==0.7.12
# !pip install -q wikipedia==1.4.0
# !pip install -q pypdf==3.12.0
# !pip install -q jq==1.4.1
# !pip install -q nltk==3.8.1
# !pip install -q tiktoken==0.4.0
# !pip install -q transformers==4.39.0 sentencepiece==0.1.99
# !pip install -q sentence-transformers==2.2.2
# !pip install -q cohere==4.11.2
# !pip install -q faiss-cpu==1.7.4



In [15]:
# pip install chromadb

## llm

In [16]:
# pip install torch

In [17]:
torch.cuda.device_count()

1

In [18]:
# !pip install transformers==4.11.3  # Adjust version as needed

In [19]:
# !pip install tensorflow keras

In [20]:
# from langchain_huggingface import HuggingFacePipeline
# from langchain import HuggingFaceHub
from transformers import pipeline
import transformers
import torch
# model_id = "meta-llama/Meta-Llama-3-8B"
# # Set Hugging Face API key
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_api_key_here"

In [21]:
# from langchain_huggingface import HuggingFaceEndpoint
# model_id = "gpt2"
# # model_id = 'distilbert/distilgpt2'
# # model_id = 'deepset/roberta-base-squad2'
# # qa_pipeline = pipeline("text-generation",  model=model_id)
# llm = HuggingFaceEndpoint(
#     repo_id=model_id, max_new_tokens=400, temperature=0.5, huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
#     model_kwargs={"max_length":2500}
# )

In [22]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

In [ ]:
from langchain.document_loaders import WikipediaLoader, OnlinePDFLoader, UnstructuredURLLoader

from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt')

from langchain.embeddings import SentenceTransformerEmbeddings

from langchain.vectorstores import Chroma



# load document
# query = "Nikola Tesla"

# docs = WikipediaLoader(query=query,
#                        load_max_docs=1,
#                        doc_content_chars_max=20_000).load()

# documents = [ docs[0].page_content ]
# metadatas = [ {"document": query} ]


# url = "https://laraveldaily.com/wp-content/uploads/2020/04/laravel-tips-2020-04.pdf"

# llm_loader = OnlinePDFLoader(url)

# pages = llm_loader.load_and_split()


# urls = [
#     "https://www.apple.com/in/support/products/faqs.html",
#     "https://www.apple.com/legal/sales-support/",
# ]
# llm_loader = UnstructuredURLLoader(urls=urls)
# llm_data = llm_loader.load()



# splitter
text_splitter = NLTKTextSplitter(chunk_size=300, chunk_overlap=50)
# tokens_chunks = text_splitter.create_documents(
#     documents,
#     metadatas=metadatas
# )


# docs_text = [ chunk.page_content for chunk in tokens_chunks ]
# docs_embeddings = embedding_llm.embed_documents(docs_text)
# query_text = "Can you list a number of Nikola Tesla's inventions?"
# query_embedding = embedding_llm.embed_query(query_text)

# vector database
# save_to_dir = "/content/wiki_chroma_db"
# vector_db = Chroma.from_documents(
#     tokens_chunks,
#     embedding_llm,
#     persist_directory=save_to_dir
# )

In [ ]:
# !pip install streamlit pyngrok

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

def get_text_chunks_langchain(text):
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs


In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# # llm = hf

# qna_template = "\n".join([
#     "Answer the next question using the provided context.",
#     "If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'",
#     "### Context:",
#     "{context}",
#     "",
#     "### Question:",
#     "{question}",
#     "",
#     "### Answer:",
# ])

# qna_prompt = PromptTemplate(
#     template=qna_template,
#     input_variables=['context', 'question'],
# )

# combine_template = "\n".join([
#     "Given intermediate contexts for a question, generate a final answer.",
#     "If the answer is not contained in the intermediate contexts, say 'NO ANSWER IS AVAILABLE'",
#     "### Summaries:",
#     "{summaries}",
#     "",
#     "### Question:",
#     "{question}",
#     "",
#     "### Final Answer:",
# ])

# combine_prompt = PromptTemplate(
#     template=combine_template,
#     input_variables=['summaries', 'question'],
# )

# map_reduce_chain = load_qa_chain(llm, chain_type="map_reduce",
#                                  return_intermediate_steps=True,
#                                  question_prompt=qna_prompt,
#                                  combine_prompt=combine_prompt)

In [ ]:
def get_answer(docs, question):

  doc_content = [ doc.page_content for doc in docs]
  metadatas = [ {"document":i} for i in range(len(docs))]
  # splitter
  text_splitter = NLTKTextSplitter(chunk_size=60, chunk_overlap=5)
  tokens_chunks = text_splitter.create_documents(
      doc_content,
      metadatas=metadatas
  )

  # embeddings
  model_name = "sentence-transformers/all-MiniLM-L6-v2"

  embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

  # vector database
  save_to_dir = "/content/wiki_chroma_db"
  vector_db = Chroma.from_documents(
      tokens_chunks,
      embedding_llm,
      persist_directory=save_to_dir
  )

  similar_docs = vector_db.similarity_search(question, k=1)


  retrieved_docs_text = [doc.page_content for doc in similar_docs]  # We only need the text of the documents
  context = "\nExtracted documents:\n"
  context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

  final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

  # Redact an answer
  final_answer = READER_LLM(final_prompt)[0]["generated_text"]

  # final_answer = map_reduce_chain({"input_documents": similar_docs,
  #                                 "question": question
  #                                 }, return_only_outputs=True)
  return final_answer

In [ ]:
docs = get_text_chunks_langchain("""
Ferrari S.p.A. (/fəˈrɑːri/; Italian: [ferˈraːri]) is an Italian luxury sports car manufacturer based in Maranello. Founded in 1939 by Enzo Ferrari (1898–1988), the company built its first car in 1940, adopted its current name in 1945, and began to produce its current line of road cars in 1947. Ferrari became a public company in 1960, and from 1963 to 2014 it was a subsidiary of Fiat S.p.A. It was spun off from Fiat's successor entity, Fiat Chrysler Automobiles, in 2016. In 2024, the Wall Street Journal described the company as having been "synonymous with opulence, meticulous craftsmanship and ridiculously fast cars for nearly a century".[6]

The company currently offers a large model range which includes several supercars, grand tourers, and one SUV. Many early Ferraris, dating to the 1950s and 1960s, count among the most expensive cars ever sold at auction. Owing to a combination of its cars, enthusiast culture, and successful licensing deals, in 2019 Ferrari was labelled the world's strongest brand by the financial consultancy Brand Finance.[7] As of May 2023, Ferrari is also one of the largest car manufacturers by market capitalisation, with a value of approximately US$52 billion.[8]

Throughout its history, the company has been noted for its continued participation in racing, especially in Formula One, where its team, Scuderia Ferrari, is the series' single oldest and most successful. Scuderia Ferrari has raced since 1929, first in Grand Prix events and later in Formula One, where since 1952 it has fielded fifteen champion drivers, won sixteen Constructors' Championships, and accumulated more race victories, 1–2 finishes, podiums, pole positions, fastest laps and points than any other team in F1 history.[9][10] Historically, Ferrari was also highly active in sports car racing, where its cars took many wins in races such as the Mille Miglia, Targa Florio and 24 Hours of Le Mans, as well as several overall victories in the World Sportscar Championship. Scuderia Ferrari fans, commonly called tifosi, are known for their passion and loyalty to the team.

History
Main article: History of Ferrari
Early history

Three Scuderia Ferrari cars in 1934, all Alfa Romeo P3s. Drivers, left to right: Achille Varzi, Louis Chiron, and Carlo Felice Trossi.
Enzo Ferrari, formerly a salesman and racing driver for Alfa Romeo, founded Scuderia Ferrari, a racing team, in 1929. Originally intended to service gentleman drivers and other amateur racers, Alfa Romeo's withdrawal from racing in 1933, combined with Enzo's connections within the company, turned Scuderia Ferrari into its unofficial representative on the track.[11] Alfa Romeo supplied racing cars to Ferrari, who eventually amassed some of the best drivers of the 1930s and won many races before the team's liquidation in 1937.[11][12]: 43

Late in 1937, Scuderia Ferrari was liquidated and absorbed into Alfa Romeo,[11] but Enzo's disagreements with upper management caused him to leave in 1939. He used his settlement to found his own company, where he intended to produce his own cars. He called the company "Auto Avio Costruzioni", and headquartered it in the facilities of the old Scuderia Ferrari;[1] due to a noncompete agreement with Alfa Romeo, the company could not use the Ferrari name for another four years. The company produced a single car, the Auto Avio Costruzioni 815, which participated in only one race before the outbreak of World War II. During the war, Enzo's company produced aircraft engines and machine tools for the Italian military; the contracts for these goods were lucrative, and provided the new company with a great deal of capital. In 1943, under threat of Allied bombing raids, the company's factory was moved to Maranello. Though the new facility was nonetheless bombed twice, Ferrari remains in Maranello to this day.[1][12]: 45–47 [13]

Under Enzo Ferrari

Ferrari's factory in the early 1960s: everything in its production line was handmade by machinists, who followed technical drawings with extreme precision.[14] Much of this work is now done by industrial robots.[15]
In 1945, Ferrari adopted its current name. Work started promptly on a new V12 engine that would power the 125 S, which was the marque's first car, and many subsequent Ferraris. The company saw success in motorsport almost as soon as it began racing: the 125 S won many races in 1947,[16][17] and several early victories, including the 1949 24 Hours of Le Mans and 1951 Carrera Panamericana, helped build Ferrari's reputation as a high-quality automaker.[18][19] Ferrari won several more races in the coming years,[9][20] and early in the 1950s its road cars were already a favourite of the international elite.[21] Ferrari produced many families of interrelated cars, including the America, Monza, and 250 series, and the company's first series-produced car was the 250 GT Coupé, beginning in 1958.[22]

In 1960, Ferrari was reorganized as a public company. It soon began searching for a business partner to handle its manufacturing operations: it first approached Ford in 1963, though negotiations fell through; later talks with Fiat, who bought 50% of Ferrari's shares in 1969, were more successful.[23][24] In the second half of the decade, Ferrari also produced two cars that upended its more traditional models: the 1967 Dino 206 GT, which was its first mass-produced mid-engined road car,[a] and the 1968 365 GTB/4, which possessed streamlined styling that modernised Ferrari's design language.[27][28] The Dino in particular was a decisive movement away from the company's conservative engineering approach, where every road-going Ferrari featured a V12 engine placed in the front of the car, and it presaged Ferrari's full embrace of mid-engine architecture, as well as V6 and V8 engines, in the 1970s and 1980s.[27]

Contemporary
Enzo Ferrari died in 1988, an event that saw Fiat expand its stake to 90%.[29] The last car that he personally approved—the F40—expanded on the flagship supercar approach first tried by the 288 GTO four years earlier.[30] Enzo was replaced in 1991 by Luca Cordero di Montezemolo, under whose 23-year-long chairmanship the company greatly expanded. Between 1991 and 2014, he increased the profitability of Ferrari's road cars nearly tenfold, both by increasing the range of cars offered and through limiting the total number produced. Montezemolo's chairmanship also saw an expansion in licensing deals, a drastic improvement in Ferrari's Formula One performance (not least through the hiring of Michael Schumacher and Jean Todt), and the production of three more flagship cars: the F50, the Enzo, and the LaFerrari. In addition to his leadership of Ferrari, Montezemolo was also the chairman of Fiat proper between 2004 and 2010.[31]

After Montezemolo resigned, he was replaced in quick succession by many new chairmen and CEOs. He was succeeded first by Sergio Marchionne,[31] who would oversee Ferrari's initial public offering and subsequent spin-off from Fiat Chrysler Automobiles,[32][33] and then by Louis Camilleri as CEO and John Elkann as chairman.[34] Beginning in 2021, Camilleri was replaced as CEO by Benedetto Vigna, who has announced plans to develop Ferrari's first fully electric model.[35] During this period, Ferrari has expanded its production, owing to a global increase in wealth, while becoming more selective with its licensing deals.[36][37]

Motorsport
Main article: Scuderia Ferrari
For a complete list of Ferrari racing cars, see List of Ferrari competition cars.
Since the company's beginnings, Ferrari has been involved in motorsport. Through its works team, Scuderia Ferrari, it has competed in a range of categories including Formula One and sports car racing, though the company has also worked in partnership with other teams.

Grand Prix and Formula One racing
Further information: Grand Prix racing history of Scuderia Ferrari and Ferrari Grand Prix results

A Ferrari F2004 Formula One car, driven by Michael Schumacher. Schumacher is one of the most decorated drivers in F1 history.
The earliest Ferrari entity, Scuderia Ferrari, was created in 1929—ten years before the founding of Ferrari proper—as a Grand Prix racing team. It was affiliated with automaker Alfa Romeo, for whom Enzo had worked in the 1920s. Alfa Romeo supplied racing cars to Ferrari, which the team then tuned and adjusted to their desired specifications. Scuderia Ferrari was highly successful in the 1930s: between 1929 and 1937 the team fielded such top drivers as Antonio Ascari, Giuseppe Campari, and Tazio Nuvolari, and won 144 out of its 225 races.[12][11]

Ferrari returned to Grand Prix racing in 1947, which was at that point metamorphosing into modern-day Formula One. The team's first homebuilt Grand Prix car, the 125 F1, was first raced at the 1948 Italian Grand Prix, where its encouraging performance convinced Enzo to continue the company's costly Grand Prix racing programme.[38]: 9  Ferrari's first victory in an F1 series was at the 1951 British Grand Prix, heralding its strong performance during the 1950s and early 1960s: between 1952 and 1964, the team took home six World Drivers' Championships and one Constructors' Championship. Notable Ferrari drivers from this era include Alberto Ascari, Juan Manuel Fangio, Phil Hill, and John Surtees.[9]

Ferrari's initial fortunes ran dry after 1964, and its began to receive its titles in isolated sprees.[10] Ferrari first started to slip in the late 1960s, when it was outclassed by teams using the inexpensive, well-engineered Cosworth DFV engine.[39][40] The team's performance improved markedly in the mid-1970s thanks to Niki Lauda, whose skill behind the wheel granted Ferrari a drivers' title in 1975 and 1977; similar success was accomplished in following years by the likes of Jody Scheckter and Gilles Villeneuve.[10][41] The team also won the Constructors' Championship in 1982 and 1983.[9][42]

Following another drought in the 1980s and 1990s, Ferrari saw a long winning streak in the 2000s, largely through the work of Michael Schumacher. After signing onto the team in 1996, Schumacher gave Ferrari five consecutive drivers' titles between 2000 and 2004; this was accompanied by six consecutive constructors' titles, beginning in 1999. Ferrari was especially dominant in the 2004 season, where it lost only three races.[9] After Schumacher's departure, Ferrari won one more drivers' title—given in 2007 to Kimi Räikkönen—and two constructors' titles in 2007 and 2008. These are the team's most recent titles to date; as of late, Ferrari has struggled to outdo recently ascendant teams such as Red Bull and Mercedes-Benz.[9][10]

Ferrari Driver Academy
Main article: Ferrari Driver Academy
Ferrari's junior driver programme is the Ferrari Driver Academy. Begun in 2009, the initiative follows the team's successful grooming of Felipe Massa between 2003 and 2006. Drivers who are accepted into the Academy learn the rules and history of formula racing as they compete, with Ferrari's support, in feeder classes such as Formula Three and Formula 4.[43][44][45] As of 2019, 5 out of 18 programme inductees had graduated and become F1 drivers: one of these drivers, Charles Leclerc, came to race for Scuderia Ferrari, while the other four signed to other teams. Non-graduate drivers have participated in racing development, filled consultant roles, or left the Academy to continue racing in lower-tier formulae.[45]

Sports car racing

A 312 P, driven by Jacky Ickx, during Ferrari's final year in the World Sportscar Championship
Aside from an abortive effort in 1940, Ferrari began racing sports cars in 1947, when the 125 S won six out of the ten races it participated in. [16] Ferrari continued to see similar luck in the years to follow: by 1957, just ten years after beginning to compete, Ferrari had won three World Sportscar Championships, seven victories in the Mille Miglia, and two victories at the 24 Hours of Le Mans, among many other races[20] These races were ideal environments for the development and promotion of Ferrari's earlier road cars, which were broadly similar to their racing counterparts.[46]

This luck continued into the first half of the 1960s, when Ferrari won the WSC's 2000GT class three consecutive times and finished first at Le Mans for six consecutive years.[47][48] Its winning streak at Le Mans was broken by Ford in 1966,[48] and though Ferrari would win two more WSC titles—one in 1967 and another in 1972[49][50]—poor revenue allocation, combined with languishing performance in Formula One, led the company to cease competing in sports car events in 1973.[24]: 621  From that point onward, Ferrari would help prepare sports racing cars for privateer teams, but would not race them itself.[51]

In 2023, Ferrari reentered sports car racing. For the 2023 FIA World Endurance Championship, Ferrari, in partnership with AF Corse, fielded two 499P sports prototypes. To commemorate the company's return to the discipline, one of the cars was numbered "50", referencing the fifty years that had elapsed since a works Ferrari competed in an endurance race.[52][53] The 499P finished first at the 2023 24 Hours of Le Mans, ending Toyota Gazoo Racing's six-year winning streak there and becoming the first Ferrari in 58 years to win the race.[54]

Other disciplines
From 1932 to 1935 Scuderia Ferrari operated a motorcycle racing division, which was conceived as a way to scout and train future Grand Prix drivers. Instead of Italian motorcycles, the team used British ones manufactured by Norton and Rudge. Though Ferrari was successful on two wheels, winning three national titles and 44 overall victories, it was eventually pushed out of the discipline both by the obsolescence of pushrod motorcycle engines and broader economic troubles stemming from the Great Depression.[55][56]

Ferrari formerly participated in a variety of non-F1 open-wheel series. As early as 1948, Ferrari had developed cars for Formula Two and Formula Libre events,[57] and the company's F2 programme led directly to the creation of the Dino engine, which came to power various racing and road Ferraris.[27] The final non-F1 formula in which Ferrari competed was the Tasman Series, wherein Chris Amon won the 1969 championship in a Dino 246 Tasmania.[58]

At least two water speed record boats have utilized Ferrari powertrains, both of them 800kg-class hydroplanes from the early 1950s. Neither boat was built by or affiliated with Ferrari, though one of them, Arno XI, had its engine order approved directly by Enzo Ferrari. Arno XI still holds the top speed record for an 800kg hydroplane
""")

In [ ]:
get_answer(docs, "when was it the most popular?")

In [ ]:
%%writefile app.py
import streamlit as st
from io import StringIO
import fitz  # PyMuPDF

# Set page title and layout
st.set_page_config(page_title="Q&A App", layout="wide")

# Title of the app
st.title("Document & URL Q&A App")

# Sidebar for document upload and URL input
st.sidebar.header("Upload Document or Enter URL")

# Option to upload a document
uploaded_file = st.sidebar.file_uploader("Choose a document...", type=["txt", "pdf"])

# Option to input a URL
url = st.sidebar.text_input("Enter a URL")

# Display the uploaded document or URL
document_content = None

if uploaded_file is not None:
    st.subheader("Uploaded Document")
    if uploaded_file.type == "text/plain":
        stringio = StringIO(uploaded_file.getvalue().decode("utf-8"))
        document_content = stringio.read()
        st.text_area("Document content:", document_content, height=300)

        docs = get_text_chunks_langchain(document_content)
    elif uploaded_file.type == "application/pdf":
        # Read PDF file
        pdf_document = fitz.open(stream=uploaded_file.read(), filetype="pdf")
        document_content = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            document_content += page.get_text()
        st.text_area("Document content:", document_content, height=300)

        docs = get_text_chunks_langchain(document_content)
    else:
        st.warning("Only .txt and .pdf files are supported at this time.")
elif url:
    st.subheader("Entered URL")
    st.write(f"URL: {url}")

    llm_loader = UnstructuredURLLoader(urls=[url])
    docs = llm_loader.load()

else:
  docs = None

# Input for user's question
st.subheader("Ask a Question")
question = st.text_input("Enter your question:")

# Placeholder for the answer
if question:
    st.subheader("Answer")
    # Placeholder where you will handle the answer logic
    if docs != None:
      answer = get_answer(docs)
      st.write("This is where the answer will be displayed.")

# Instructions or notes
st.sidebar.info(
    """
    Please upload a document or enter a URL for the content you want to ask questions about.
    Then, enter your question in the main area.
    """
)

# Add a footer
st.markdown("""
    <style>
        footer {
            visibility: hidden;
        }
        footer:after {
            content:'Q&A App by YourName';
            visibility: visible;
            display: block;
            position: relative;
            padding: 10px;
            top: 2px;
        }
    </style>
    """, unsafe_allow_html=True)


In [ ]:
from pyngrok import ngrok
import time
import subprocess

# Kill any existing ngrok processes
ngrok.kill()

# Start the Streamlit app
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Allow some time for the app to start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(f'Public URL: {public_url}')
